# Read Data

In [1]:
import torch
import numpy as np
import pandas as pd

In [2]:
train_balanced = pd.read_pickle('train_balanced.pkl')
val_balanced = pd.read_pickle('val_balanced.pkl')
test_balanced = pd.read_pickle('test_balanced.pkl')
VOCAB_BALANCED_SIZE = set()
for tokenized in train_balanced.tokenized:
    for token in tokenized:
        VOCAB_BALANCED_SIZE.add(token)
max(VOCAB_BALANCED_SIZE)     

7130

In [3]:
train_unbalanced = pd.read_pickle('train_unbalanced.pkl')
val_unbalanced = pd.read_pickle('val_unbalanced.pkl')
test_unbalanced = pd.read_pickle('test_unbalanced.pkl')

VOCAB_UNBALANCED_SIZE = set()
for tokenized in train_unbalanced.tokenized:
    for token in tokenized:
        VOCAB_UNBALANCED_SIZE.add(token)

max(VOCAB_UNBALANCED_SIZE)

6767

In [4]:
from torch.utils.data import Dataset, DataLoader
class ClassificationDataset(Dataset):
    def __init__(self, data, X, y):
        self.X = data[X]
        self.y = data[y]
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return torch.tensor(self.X[idx], dtype=torch.float), torch.tensor(self.y[idx], dtype=torch.float)


In [7]:
y_unbalanced=np.stack(train_unbalanced['MultiLabel'].to_numpy())
y_balanced=np.stack(train_balanced['MultiLabel'].to_numpy())


In [8]:
train_dataset_embed_balanced = ClassificationDataset(train_balanced, "precomputed_embeddings", "MultiLabel")
val_dataset_embed_balanced = ClassificationDataset(val_balanced, "precomputed_embeddings", "MultiLabel")
test_dataset_embed_balanced= ClassificationDataset(test_balanced, "precomputed_embeddings", "MultiLabel")

train_loader_balanced = DataLoader(train_dataset_embed_balanced, batch_size=16, shuffle=True)
val_loader_balanced = DataLoader(val_dataset_embed_balanced, batch_size=16, shuffle=False)
test_loader_balanced = DataLoader(test_dataset_embed_balanced, batch_size=16, shuffle=False)

In [9]:
train_dataset_embed_unbalanced = ClassificationDataset(train_unbalanced, "precomputed_embeddings", "MultiLabel")
val_dataset_embed_unbalanced = ClassificationDataset(val_unbalanced, "precomputed_embeddings", "MultiLabel")
test_dataset_embed_unbalanced= ClassificationDataset(test_unbalanced, "precomputed_embeddings", "MultiLabel")

train_loader_unbalanced = DataLoader(train_dataset_embed_unbalanced, batch_size=16, shuffle=True)
val_loader_unbalanced = DataLoader(val_dataset_embed_unbalanced, batch_size=16, shuffle=False)
test_loader_unbalanced = DataLoader(test_dataset_embed_unbalanced, batch_size=16, shuffle=False)

# RNN

In [10]:
from torch import nn
from torch.optim import Adam
from torch.functional import F
from sklearn.metrics import f1_score

In [11]:
class MultiLabelLSTM(nn.Module):
    def __init__(self,
              vocab_size,
              embedding_dim,
              hidden_dim,
              num_layers,
              num_classes,
              bidirectional,
              dropout,
    ):
        super().__init__()
        # LSTM Layer
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers, bidirectional=bidirectional, dropout=dropout)
        # Linear Layer
        if bidirectional:
            self.linear = nn.Linear(hidden_dim * 2, num_classes)
        else:
            self.linear = nn.Linear(hidden_dim, num_classes)
    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        linear = self.linear(lstm_out)
        return linear

In [12]:
!nvidia-smi

Sat May 10 21:39:29 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.133.07             Driver Version: 572.83         CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 2080 Ti     On  |   00000000:01:00.0  On |                  N/A |
| 31%   41C    P2             74W /  260W |    1269MiB /  11264MiB |     27%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Train & Validate 

In [13]:
import optuna
import torch
import itertools
from torch.nn.functional import sigmoid
from tqdm import tqdm
from torcheval.metrics.functional import multiclass_f1_score

In [14]:
def get_prediction(row):
    row = sigmoid(torch.tensor(row))
    row[row >=0.5] = 1
    row[row <0.5] = 0
    row.numpy().astype(np.float32)
    return row

In [21]:
# for weighted loss
pos_weight_balanced = (y_balanced.shape[0] - y_balanced.sum(axis=0)) / (y_balanced.sum(axis=0) + 0.0000001)
pos_weight_balanced = torch.tensor(pos_weight_balanced, dtype=torch.float32).to("cuda")  # or your device

def objective_balanced(trial, epochs=3):
    # Hyperparameter search space
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    embedding_dim = trial.suggest_categorical("embedding_dim", [768])
    hidden_dim = trial.suggest_categorical("hidden_dim", [64, 128, 256, 512])
    num_layers = trial.suggest_int("num_layers", 2, 5, step=1)
    bidirectional = trial.suggest_categorical("bidirectional", [True, False])
    dropout = trial.suggest_categorical("dropout", [0.1, 0.25, 0.5])
    NUM_CLASSES = 4
    MAX_LEN = 1

    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    model = MultiLabelLSTM(
        vocab_size=max(VOCAB_BALANCED_SIZE),
        embedding_dim=embedding_dim,
        hidden_dim=hidden_dim,
        num_layers=num_layers,
        num_classes=NUM_CLASSES,
        bidirectional=bidirectional,
        dropout=dropout,
    ).to(DEVICE)
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight_balanced)
    optimizer = Adam(model.parameters(), lr=lr)
    
	
    model.train()
    for epoch in range(epochs):
        for X_batch, y_batch in train_loader_balanced:
            X_batch, y_batch = X_batch.to(DEVICE), y_batch.to(DEVICE)
            optimizer.zero_grad()
            y_preds = model(X_batch)
            loss = criterion(y_preds, y_batch)
            loss.backward()
            optimizer.step()
    # Validation
    model.eval()
    y_preds_list = []
    y_true_list = []
    with torch.no_grad():
        for X_batch, y_batch in val_loader_balanced:
            X_batch, y_batch = X_batch.to(DEVICE), y_batch.to(DEVICE)
            y_preds = model(X_batch)
            y_preds_list.extend(y_preds.detach().cpu().numpy())
            y_true_list.extend(y_batch.detach().cpu().numpy())
    y_preds = np.array([get_prediction(pred) for pred in y_preds_list])
    y_true = np.array(y_true_list)

    f1 = f1_score(y_true, y_preds, average='weighted')
    return f1


In [22]:
# ---- Run the Optuna Study ----
study_balanced = optuna.create_study(direction="maximize")
study_balanced.optimize(objective_balanced, n_trials=15)

print("Best hyperparameters:", study_balanced.best_trial.params)

[I 2025-05-10 21:43:37,493] A new study created in memory with name: no-name-2f61b9aa-b4ac-4ee9-b0d6-cd4cca40d780
[I 2025-05-10 21:43:40,365] Trial 0 finished with value: 0.712322495675605 and parameters: {'embedding_dim': 768, 'hidden_dim': 128, 'num_layers': 4, 'bidirectional': True, 'dropout': 0.1, 'lr': 0.0009879073621631908}. Best is trial 0 with value: 0.712322495675605.
[I 2025-05-10 21:43:42,061] Trial 1 finished with value: 0.7022927433959593 and parameters: {'embedding_dim': 768, 'hidden_dim': 128, 'num_layers': 4, 'bidirectional': False, 'dropout': 0.1, 'lr': 0.0012843821699711861}. Best is trial 0 with value: 0.712322495675605.
[I 2025-05-10 21:43:44,250] Trial 2 finished with value: 0.6284817088860427 and parameters: {'embedding_dim': 768, 'hidden_dim': 128, 'num_layers': 5, 'bidirectional': True, 'dropout': 0.25, 'lr': 0.002404467085282438}. Best is trial 0 with value: 0.712322495675605.
[I 2025-05-10 21:43:46,464] Trial 3 finished with value: 0.7409870341410524 and param

Best hyperparameters: {'embedding_dim': 768, 'hidden_dim': 512, 'num_layers': 2, 'bidirectional': True, 'dropout': 0.1, 'lr': 0.0002049173817318604}


In [29]:
# for weighted loss
pos_weight_unbalanced = (y_unbalanced.shape[0] - y_unbalanced.sum(axis=0)) / (y_balanced.sum(axis=0) + 0.0000001)
pos_weight_unbalanced = torch.tensor(pos_weight_unbalanced, dtype=torch.float32).to("cuda")  # or your device

def objective_unbalanced(trial, epochs=3):
    # Hyperparameter search space
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    embedding_dim = trial.suggest_categorical("embedding_dim", [768])
    hidden_dim = trial.suggest_categorical("hidden_dim", [64, 128, 256, 512])
    num_layers = trial.suggest_int("num_layers", 2, 5, step=1)
    bidirectional = trial.suggest_categorical("bidirectional", [True, False])
    dropout = trial.suggest_categorical("dropout", [0.1, 0.25, 0.5])
    NUM_CLASSES = 4
    MAX_LEN = 1

    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    model = MultiLabelLSTM(
        vocab_size=len(VOCAB_UNBALANCED_SIZE) + 1000,
        embedding_dim=embedding_dim,
        hidden_dim=hidden_dim,
        num_layers=num_layers,
        num_classes=NUM_CLASSES,
        bidirectional=bidirectional,
        dropout=dropout,
    ).to(DEVICE)
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight_unbalanced)
    optimizer = Adam(model.parameters(), lr=lr)
    
	
    model.train()
    for epoch in range(epochs):
        for X_batch, y_batch in train_loader_unbalanced:
            X_batch, y_batch = X_batch.to(DEVICE), y_batch.to(DEVICE)
            optimizer.zero_grad()
            y_preds = model(X_batch)
            loss = criterion(y_preds, y_batch)
            loss.backward()
            optimizer.step()
    # Validation
    model.eval()
    y_preds_list = []
    y_true_list = []
    with torch.no_grad():
        for X_batch, y_batch in val_loader_unbalanced:
            X_batch, y_batch = X_batch.to(DEVICE), y_batch.to(DEVICE)
            y_preds = model(X_batch)
            y_preds_list.extend(y_preds.detach().cpu().numpy())
            y_true_list.extend(y_batch.detach().cpu().numpy())
    y_preds = np.array([get_prediction(pred) for pred in y_preds_list])
    y_true = np.array(y_true_list)

    f1 = f1_score(y_true, y_preds, average='weighted')
    return f1


In [30]:
# ---- Run the Optuna Study ----
study_unbalanced = optuna.create_study(direction="maximize")
study_unbalanced.optimize(objective_unbalanced, n_trials=15)

print("Best hyperparameters:", study_unbalanced.best_trial.params)

[I 2025-05-10 21:48:35,220] A new study created in memory with name: no-name-8ef45d91-19f6-4a7b-999e-0a2c3713ec96
[I 2025-05-10 21:48:38,025] Trial 0 finished with value: 0.6116827073281391 and parameters: {'embedding_dim': 768, 'hidden_dim': 256, 'num_layers': 4, 'bidirectional': True, 'dropout': 0.5, 'lr': 0.0023946223363255693}. Best is trial 0 with value: 0.6116827073281391.
[I 2025-05-10 21:48:40,125] Trial 1 finished with value: 0.5622926518168503 and parameters: {'embedding_dim': 768, 'hidden_dim': 512, 'num_layers': 4, 'bidirectional': False, 'dropout': 0.25, 'lr': 0.004538281873184708}. Best is trial 0 with value: 0.6116827073281391.
[I 2025-05-10 21:48:41,354] Trial 2 finished with value: 0.5642724934059048 and parameters: {'embedding_dim': 768, 'hidden_dim': 128, 'num_layers': 4, 'bidirectional': False, 'dropout': 0.5, 'lr': 0.0003255612117100354}. Best is trial 0 with value: 0.6116827073281391.
[I 2025-05-10 21:48:42,986] Trial 3 finished with value: 0.6831403170613638 and 

Best hyperparameters: {'embedding_dim': 768, 'hidden_dim': 256, 'num_layers': 2, 'bidirectional': True, 'dropout': 0.25, 'lr': 0.00025457032231386887}


In [31]:
def train_val(
        model: MultiLabelLSTM,
        optim: Adam,
        criterion: nn.BCEWithLogitsLoss,
        epochs: int,
        train_dataloader: DataLoader,
        val_dataloader: DataLoader,
        device
    ):
    best_f1 = 0
    best_model = None
    model.to(device)
    for epoch in tqdm(range(epochs)):
        model.train()
        train_loss = 0
        y_preds_list_train = []
        y_true_list_train = []
        for train_X, train_y in train_dataloader:
            train_X, train_y = train_X.to(device), train_y.to(device)
    
            y_preds = model(train_X)
            loss = criterion(y_preds, train_y)
    
            optim.zero_grad()
            loss.backward()
            optim.step()
    
            train_loss += loss.item()
            y_preds_list_train.extend(y_preds.detach().cpu().numpy())
            y_true_list_train.extend(train_y.detach().cpu().numpy())
    
        y_preds = np.array([get_prediction(pred) for pred in y_preds_list_train])
        y_true = np.array(y_true_list_train) 
        train_f1 =f1_score(y_true, y_preds, average='weighted')
        # Validation
        model.eval()
        val_loss = 0
        y_preds_list_val = []
        y_true_list_val = []
        with torch.no_grad():
            for val_X, val_y in val_dataloader:
                val_X, val_y = val_X.to(device), val_y.to(device)
    
                y_preds = model(val_X)
                loss = criterion(y_preds, val_y)
                val_loss += loss.item()
                predicted = torch.argmax(y_preds, dim=1)
                y_preds_list_val.extend(y_preds.detach().cpu().numpy())
                y_true_list_val.extend(val_y.detach().cpu().numpy())
    
        y_preds = np.array([get_prediction(pred) for pred in y_preds_list_val])
        y_true = np.array(y_true_list_val)
        val_f1 = f1_score(y_true, y_preds, average='weighted')
        if val_f1 > best_f1:
            best_f1 = val_f1
            best_model = model.state_dict()
        print(
            f"Epoch {epoch+1}/{epochs}, "
            f"Train Loss: {train_loss/len(train_dataloader):.4f}, "
            f"Val Loss: {val_loss/len(val_dataloader):.4f}, "
            f"Train F1: {train_f1:.2f}%, "
            f"Val F1: {val_f1:.2f}%"
        )
    return best_model

In [41]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
model_balanced = MultiLabelLSTM(
			vocab_size=len(VOCAB_BALANCED_SIZE),
			embedding_dim=study_balanced.best_trial.params['embedding_dim'],
			hidden_dim=study_balanced.best_trial.params['hidden_dim'],
			num_layers=study_balanced.best_trial.params['num_layers'],
			num_classes=4,
			bidirectional=study_balanced.best_trial.params['bidirectional'],
            dropout = study_balanced.best_trial.params['dropout'],
		).to(DEVICE)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight_balanced)
optimizer_balanced = Adam(model_balanced.parameters(), lr=study_balanced.best_trial.params['lr'])
params_balanced = train_val(
    model_balanced,
    optimizer_balanced,
    criterion,
    epochs = 10,
    train_dataloader = train_loader_balanced,
    val_dataloader = val_loader_balanced,
    device=DEVICE
)

 10%|████████▍                                                                           | 1/10 [00:01<00:12,  1.40s/it]

Epoch 1/10, Train Loss: 0.6984, Val Loss: 0.5882, Train F1: 0.60%, Val F1: 0.72%


 20%|████████████████▊                                                                   | 2/10 [00:02<00:11,  1.46s/it]

Epoch 2/10, Train Loss: 0.5880, Val Loss: 0.5404, Train F1: 0.69%, Val F1: 0.76%


 30%|█████████████████████████▏                                                          | 3/10 [00:04<00:09,  1.41s/it]

...

Epoch 8/10, Train Loss: 0.2852, Val Loss: 0.7854, Train F1: 0.86%, Val F1: 0.74%


 90%|███████████████████████████████████████████████████████████████████████████▌        | 9/10 [00:12<00:01,  1.39s/it]

Epoch 9/10, Train Loss: 0.2501, Val Loss: 0.9017, Train F1: 0.87%, Val F1: 0.73%


100%|███████████████████████████████████████████████████████████████████████████████████| 10/10 [00:14<00:00,  1.41s/it]

Epoch 10/10, Train Loss: 0.2193, Val Loss: 1.0310, Train F1: 0.89%, Val F1: 0.74%


In [40]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
model_unbalanced = MultiLabelLSTM(
			vocab_size=len(VOCAB_UNBALANCED_SIZE)+1000,
			embedding_dim=study_unbalanced.best_trial.params['embedding_dim'],
			hidden_dim=study_unbalanced.best_trial.params['hidden_dim'],
			num_layers=study_unbalanced.best_trial.params['num_layers'],
			num_classes=4,
			bidirectional=study_unbalanced.best_trial.params['bidirectional'],
            dropout = study_unbalanced.best_trial.params['dropout'],
		).to(DEVICE)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight_unbalanced)
optimizer_unbalanced = Adam(model_balanced.parameters(), lr=study_unbalanced.best_trial.params['lr'])
params_unbalanced = train_val(
    model_unbalanced,
    optimizer_unbalanced,
    criterion,
    epochs = 10,
    train_dataloader = train_loader_unbalanced,
    val_dataloader = val_loader_unbalanced,
    device=DEVICE
)

 10%|████████▍                                                                           | 1/10 [00:00<00:06,  1.34it/s]

Epoch 1/10, Train Loss: 0.6701, Val Loss: 0.6719, Train F1: 0.42%, Val F1: 0.40%


 20%|████████████████▊                                                                   | 2/10 [00:01<00:06,  1.33it/s]

Epoch 2/10, Train Loss: 0.6705, Val Loss: 0.6719, Train F1: 0.42%, Val F1: 0.40%


 30%|█████████████████████████▏                                                          | 3/10 [00:02<00:05,  1.37it/s]

...

Epoch 8/10, Train Loss: 0.6709, Val Loss: 0.6719, Train F1: 0.42%, Val F1: 0.40%


 90%|███████████████████████████████████████████████████████████████████████████▌        | 9/10 [00:06<00:00,  1.40it/s]

Epoch 9/10, Train Loss: 0.6701, Val Loss: 0.6719, Train F1: 0.43%, Val F1: 0.40%


100%|███████████████████████████████████████████████████████████████████████████████████| 10/10 [00:07<00:00,  1.39it/s]

Epoch 10/10, Train Loss: 0.6702, Val Loss: 0.6719, Train F1: 0.42%, Val F1: 0.40%


# Test

In [52]:
from sklearn.metrics import classification_report, multilabel_confusion_matrix
from sklearn.metrics import accuracy_score

In [44]:
def testing(
		model: MultiLabelLSTM,
		criterion: nn.BCEWithLogitsLoss,
		test_dataloader: DataLoader,
		device
    ):
    model.to(device)
    model.eval()
    test_loss = 0
    y_preds_list = []
    y_true_list = []
    with torch.no_grad():
        for test_X, test_y in test_dataloader:
            test_X, test_y = test_X.to(device), test_y.to(device)
    
            y_preds = model(test_X)
            y_preds_list.extend(y_preds.detach().cpu().numpy())
            y_true_list.extend(test_y.detach().cpu().numpy())
            loss = criterion(y_preds, test_y)
            test_loss += loss.item()
        
        y_preds_list = [get_prediction(pred) for pred in y_preds_list]
    print(
        f"test Loss: {test_loss/len(test_dataloader):.4f}, "
        f"test Acc: {accuracy_score(y_true_list, y_preds_list)}%"
    )
    return np.array(y_preds_list), np.array(y_true_list)

In [47]:
best_model_balanced = MultiLabelLSTM(
			vocab_size=len(VOCAB_BALANCED_SIZE),
			embedding_dim=study_balanced.best_trial.params['embedding_dim'],
			hidden_dim=study_balanced.best_trial.params['hidden_dim'],
			num_layers=study_balanced.best_trial.params['num_layers'],
			num_classes=4,
			bidirectional=study_balanced.best_trial.params['bidirectional'],
            dropout = study_balanced.best_trial.params['dropout'],
		).to(DEVICE)
best_model_balanced.load_state_dict(params_balanced)
y_preds, y_true=testing(best_model_balanced, criterion, test_loader_balanced, 'cuda')


test Loss: 1.0777, test Acc: 0.39928698752228164%


In [54]:
print(classification_report(y_true=y_true, y_pred=y_preds))
print(multilabel_confusion_matrix(y_true=y_true, y_pred=y_preds))

              precision    recall  f1-score   support

           0       0.78      0.79      0.79       229
           1       0.63      0.59      0.61       215
           2       0.84      0.83      0.84       387
           3       0.57      0.44      0.50       151

   micro avg       0.75      0.71      0.73       982
   macro avg       0.71      0.66      0.68       982
weighted avg       0.74      0.71      0.72       982
 samples avg       0.79      0.77      0.74       982



/home/malik/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


array([[[280,  52],
        [ 47, 182]],

       [[273,  73],
        [ 89, 126]],

       [[113,  61],
        [ 65, 322]],

       [[361,  49],
        [ 85,  66]]])

In [55]:
best_model_unbalanced = MultiLabelLSTM(
			vocab_size=len(VOCAB_UNBALANCED_SIZE)+1000,
			embedding_dim=study_unbalanced.best_trial.params['embedding_dim'],
			hidden_dim=study_unbalanced.best_trial.params['hidden_dim'],
			num_layers=study_unbalanced.best_trial.params['num_layers'],
			num_classes=4,
			bidirectional=study_unbalanced.best_trial.params['bidirectional'],
            dropout = study_unbalanced.best_trial.params['dropout'],
		).to(DEVICE)
best_model_unbalanced.load_state_dict(params_unbalanced)
y_preds, y_true=testing(best_model_unbalanced, criterion, test_loader_unbalanced, 'cuda')


test Loss: 0.7670, test Acc: 0.03208556149732621%


In [56]:
print(classification_report(y_true=y_true, y_pred=y_preds))
print(multilabel_confusion_matrix(y_true=y_true, y_pred=y_preds))

              precision    recall  f1-score   support

           0       0.48      0.14      0.22       229
           1       0.39      0.73      0.51       215
           2       0.71      0.29      0.41       387
           3       0.25      0.67      0.36       151

   micro avg       0.39      0.41      0.40       982
   macro avg       0.46      0.46      0.37       982
weighted avg       0.52      0.41      0.38       982
 samples avg       0.35      0.40      0.34       982

[[[298  34]
  [197  32]]

 [[101 245]
  [ 59 156]]

 [[128  46]
  [275 112]]

 [[104 306]
  [ 50 101]]]


/home/malik/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
